# Cheese Recommender

## EDA and Cleaning

In [14]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import unicodedata

Recommending cheese based on blah blah blah...  Cheese.com very lenient API

In [15]:
# took Adi Bronshtein's function for conducting exploratory data analysis and added spaces between each eda test
# using blank print statesments and \n for easier readability
def eda(dataframe):
    print("missing values: \n{}".format(dataframe.isnull().sum()))
    print()
    print("dataframe index: \n{}".format(dataframe.index))
    print()
    print("dataframe types: \n{}".format(dataframe.dtypes))
    print()
    print("dataframe shape: \n{}".format(dataframe.shape))
    print()
    print("dataframe describe: \n{}".format(dataframe.describe()))
    print()

    print("number of unique values in each column: ")
    for item in dataframe:
        print(item)
        print(dataframe[item].nunique())

In [16]:
#read fork_and_bottle.csv into 'chz_fork'
chz_fork = pd.read_csv('./data/fork_and_bottle.csv')


In [17]:
# function to strip strings of accents, from BartoszKP and oefe on StackOverflow
# https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-in-a-python-unicode-string

import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')


In [18]:
chz_fork

,Name,Milk,Region,Description
0,Abbaye de Belloc,sheep,FR,From the Pyrenees. Probably the cheese most re...
1,Abbaye de Citeaux,cow/R,FR,Produced by the monks of the Abbey of Saint Ni...
2,Abbaye de Tamie,cow,FR,Similar to Reblochon but larger - a pressed un...
3,Acappella,goat,"CA, US","Andante in Petaluma, CA. A small cheese with a..."
4,Affidelice,cow,FR,Made by Berthaudt in Burgundy. A washed rind c...
5,Aged Chelsea,goat,"MI, US",From Zingerman's Creamery. Mold ripened and ag...
6,Alpage Prattigau,cow,CH,A Rolf Beeler selection. Made from milk of cow...
7,Alta Robiola Bosina,c-s,IT,Cow and sheep's milk from Langhe/Piedmonte reg...
8,Amish Frolic,cow/r,"NJ, US",Made by a collaboration between Bobolink Dairy...
9,Appenzellar,cow,CH,"Gold or Black label??? Very nutty, spicy fin..."


In [19]:
eda(chz_fork)

missing values: 
Name           0
Milk           1
Region         0
Description    0
dtype: int64

dataframe index: 
RangeIndex(start=0, stop=365, step=1)

dataframe types: 
Name           object
Milk           object
Region         object
Description    object
dtype: object

dataframe shape: 
(365, 4)

dataframe describe: 
                         Name Milk Region              Description
count                     365  364    365                      365
unique                    362   34     43                      357
top     Caciotta al Rosmarino  cow     FR  From Pascal Beillevaire
freq                        2  136    124                        9

number of unique values in each column: 
Name
362
Milk
34
Region
43
Description
357


In [20]:
chz_fork[chz_fork['Milk'].isnull()]

,Name,Milk,Region,Description
211,Moncenisio,NaN,IT,see Bleu de Moncenisio


In [21]:
# according to https://culturecheesemag.com/cheese-library/blu-del-moncenisio, Moncenisio is made from cow milk
chz_fork[chz_fork['Name'] == 'Moncenisio'] = chz_fork[chz_fork['Name'] == 'Moncenisio'].fillna('cow')

In [22]:
chz_fork[chz_fork['Name'] == 'Moncenisio']

,Name,Milk,Region,Description
211,Moncenisio,cow,IT,see Bleu de Moncenisio


In [23]:
eda(chz_fork)

missing values: 
Name           0
Milk           0
Region         0
Description    0
dtype: int64

dataframe index: 
RangeIndex(start=0, stop=365, step=1)

dataframe types: 
Name           object
Milk           object
Region         object
Description    object
dtype: object

dataframe shape: 
(365, 4)

dataframe describe: 
                         Name Milk Region              Description
count                     365  365    365                      365
unique                    362   34     43                      357
top     Caciotta al Rosmarino  cow     FR  From Pascal Beillevaire
freq                        2  137    124                        9

number of unique values in each column: 
Name
362
Milk
34
Region
43
Description
357


In [24]:
for i in chz_fork['Name']:
    if "(" in i:
        print(i)

Comte (badoz tunnel reserve)
Comté (Comte)
Gruyère (Gruyere)
Piave - Aged (Stravecchio/Vecchio)
Red Spruce (Cheddar)
Stanser Schafkase (Stanser Schafchäs)


In [25]:
chz_fork['Milk'].unique()

array(['sheep', 'cow/R', 'cow', 'goat', 'c-s', 'cow/r', 'sheep/r',
       'c or g', 'wbuff', 'goat/p', 'goat/r', 'cow/p', 'sheep/R', 'c/g/s',
       'g-s/r', 'c-g', 'c/R&P', 'cow/P', 'sheep/p', 'goat/R', 'c-g-s',
       'g-s', 'c-g/p', 'goat*', 'Sheep', 'g-c-s', 'c-gt/p', 'g-c/p',
       'cow?', 'cows', 'g-c', 'goat/t', 'c/g/r', 'cow/sh'], dtype=object)

In [26]:
chz_fork[chz_fork['Milk'] == 'c or g']

,Name,Milk,Region,Description
21,Bethmale,c or g,FR,From the Pyrenees. Farmhouse cheese - rind has...


In [27]:
chz_fork['Description'][21]

'From the Pyrenees. Farmhouse cheese - rind has an leathery appearance. A mellow cheese providing a fruity spicy flavor and a somewhat pungent aroma which becomes stronger with age.'

In [28]:
# according to https://www.fromages.com/en/cheese/473-bethmale, this is a cow's milk cheese, raw, 
# pressed semi-hard cheese aged 2-3 months
chz_fork[chz_fork['Name'] == 'Bethmale'] = chz_fork[chz_fork['Name'] == 'Bethmale'].replace('c or g','cow')

In [29]:
chz_fork[chz_fork['Milk'] == 'c or g']

,Name,Milk,Region,Description


In [32]:
chz_fork[chz_fork['Name'] == 'Bethmale'] 

,Name,Milk,Region,Description
21,Bethmale,cow,FR,From the Pyrenees. Farmhouse cheese - rind has...


In [33]:
chz_fork['Treatment'] = ''

In [34]:
chz_fork['Treatment'][21] = 'raw'

In [35]:
chz_fork[chz_fork['Name'] == 'Bethmale'] 

,Name,Milk,Region,Description,Treatment
21,Bethmale,cow,FR,From the Pyrenees. Farmhouse cheese - rind has...,raw


In [40]:
chz_fork[chz_fork['Milk'] == 'goat*']

,Name,Milk,Region,Description,Treatment
209,Minuet,goat*,"CA, US",A wonderful triple crème goat cheese from Anda...,


In [41]:
# this says its a goat + cow cheese
chz_fork['Description'][209]

"A wonderful triple crème goat cheese from Andante Dairy in Santa Rosa. which is enriched with cow's milk crème fraiche from Kendall Farms."

In [42]:
chz_fork[chz_fork['Name'] == 'Minuet'] = chz_fork[chz_fork['Name'] == 'Minuet'].replace('goat*','g-c')